In [1]:
!pip uninstall dapi --yes
!pip install git+https://github.com/DesignSafe-CI/dapi.git@dev --user

Found existing installation: dapi 1.0.0
Uninstalling dapi-1.0.0:
  Successfully uninstalled dapi-1.0.0
  Cloning https://github.com/DesignSafe-CI/dapi.git (to revision dev) to /tmp/pip-req-build-t2tf2kfh
  Running command git clone --filter=blob:none --quiet https://github.com/DesignSafe-CI/dapi.git /tmp/pip-req-build-t2tf2kfh
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch 'dev' set up to track remote branch 'dev' from 'origin'.
  Resolved https://github.com/DesignSafe-CI/dapi.git to commit 18ce6e9aab54b384a56562cca06c0b41e1b214e7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dapi: filename=dapi-1.0.0-py3-none-any.whl size=24452 sha256=21957f23bdd5ea81579c6ff58e527df89a3966f840806e1f20fe56adcd99d3fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-qjp0xj3e/wheels/2e/e4/4e/ee98b2c2d7ca1faaf2d93d38772bece5f20c0e2734c00d8da0
Succ

In [2]:
import os

# Import only DSClient and exceptions needed at top level
from dapi import (
    DSClient,
    SubmittedJob,
    interpret_job_status,
    AppDiscoveryError,
    FileOperationError,
    JobSubmissionError,
    JobMonitorError,
    # Optionally import status constants if you want to check against them explicitly
    STATUS_TIMEOUT,
    STATUS_UNKNOWN,
    TAPIS_TERMINAL_STATES,
)
import json
from datetime import datetime
from dataclasses import asdict
import pandas as pd
import tqdm as notebook_tqdm

In [3]:
try:
    print("Initializing DSClient...")
    ds = DSClient()
    print("DSClient initialized.")
except Exception as e:
    print(f"Initialization failed: {e}")
    raise SystemExit("Stopping notebook due to client initialization failure.")

Initializing DSClient...


Enter DesignSafe Username:  kks32
Enter DesignSafe Password:  ········


Authentication successful.
DatabaseAccessor initialized. Connections will be created on first access.
DSClient initialized.


In [11]:
cur_dir = os.getcwd()
input_uri = ds.files.translate_path_to_uri(cur_dir + '/DS_input')
print(cur_dir, input_uri)

Translated '/home/jupyter/MyData/OpenSees-T3/DS_input' to 'tapis://designsafe.storage.default/kks32/OpenSees-T3/DS_input' using t.username
/home/jupyter/MyData/OpenSees-T3 tapis://designsafe.storage.default/kks32/OpenSees-T3/DS_input


In [16]:
max_job_minutes: int = 10
# queue: str = "skx" # Example override - only if needed and valid
tacc_allocation: str = "BCS20003"
app_id_to_use = "opensees-mp-s3"  # Or "mpm-s3" - use the correct ID verified on DesignSafe
input_filename = 'main.tcl'

In [17]:
try:
    print("\nGenerating job request dictionary...")
    job_dict = ds.jobs.generate_request(
        app_id=app_id_to_use,
        input_dir_uri=input_uri,
        script_filename=input_filename,
        max_minutes=max_job_minutes,
        allocation=tacc_allocation,
    )
    print("\n--- Generated Job Request Dictionary ---")
    print(json.dumps(job_dict, indent=2, default=str))
    print("---------------------------------------")
except (AppDiscoveryError, ValueError, JobSubmissionError) as e:
    print(f"Error generating job request: {e}")
    raise SystemExit("Stopping notebook due to job request generation error.")
except Exception as e:
    print(f"An unexpected error occurred during job request generation: {e}")
    raise SystemExit("Stopping notebook due to unexpected generation error.")


Generating job request dictionary...
Generating job request for app 'opensees-mp-s3'...
Using App Details: opensees-mp-s3 vlatest
Placing script 'main.tcl' in appArgs: 'Main Script'
Adding allocation: BCS20003
Job request dictionary generated successfully.

--- Generated Job Request Dictionary ---
{
  "name": "opensees-mp-s3-20250429_133659",
  "appId": "opensees-mp-s3",
  "appVersion": "latest",
  "description": "Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx",
  "nodeCount": 2,
  "coresPerNode": 48,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/OpenSees-T3/DS_input",
      "autoMountLocal": true,
      "targetPath": "inputDirector